Cohen manuscript zenodo data have already been downloaded by the cohen retina preprocess script.


In [2]:
data_root="/home/mcn26/palmer_scratch/tabula_data"
output_root="/home/mcn26/palmer_scratch/tabula_data/formatted"
zenodo_data_root=f"{data_root}/scMPRA/scMPRA/data"

In [3]:
import pandas as pd

In [4]:
k562_root=f"{zenodo_data_root}/sc_expression_data"

As before, there are two replicate files, but also an additional large file?

In [5]:
mystery=pd.read_csv(f"{k562_root}/k562_pBC_exp.csv")

In [41]:
rep1=pd.read_csv(f"{k562_root}/k562_pBC_exp_rep1.csv")

In [42]:
rep2=pd.read_csv(f"{k562_root}/k562_pBC_exp_rep2.csv")

In [8]:
len(rep1)

519256

In [9]:
len(rep2)

479572

In [10]:
len(mystery)

998828

Based on the sizes, it seems like the 'big file' is a merge of the two replicate files. But without replicate information it's useless.

In [11]:
rep1["rep_id"]=1
rep2["rep_id"]=2
df=pd.concat([rep1,rep2],axis=0,ignore_index=True)

In [12]:
df

,cellBC,prom_id,pBC,norm_exp,direct_exp,rep_id
0,GGGACAACAGGAGGAG,chr1_169599292_169599425_-,GAAGACTGTGTA,704.845815,10,1
1,GGGACAACAGGAGGAG,chrX_55744115_55744248_+,AGGAGGAACACA,281.938326,4,1
2,GGGACAACAGGAGGAG,chr1_229569774_229569907_-,ACCAGCGTAAGC,5004.405286,71,1
3,GGGACAACAGGAGGAG,chr17_10325195_10325328_-,AAGAAGAGTAGG,775.330396,11,1
4,GGGACAACAGGAGGAG,chr15_41221471_41221604_+,CATATACCGCTA,845.814978,12,1
...,...,...,...,...,...,...
998823,TTATTGCCACAACGCC,chr8_101162797_101162930_+,CTAGCTCATGCA,96.498054,1,2
998824,TTATTGCCACAACGCC,chr19_52490015_52490148_-,CTTAGAGCTTCA,96.498054,1,2
998825,TTATTGCCACAACGCC,chr6_42928425_42928558_+,AGCATTACCGTT,96.498054,1,2
998826,TTATTGCCACAACGCC,chr9_19380167_19380300_-,TACCATCTGAAC,96.498054,1,2


See mixed cell preprocessing for more explanations : it was done first and on an identical file-format.

In [13]:
df=df.drop(["pBC","norm_exp"],axis=1)
df=df.rename({'prom_id':'CRE_id','direct_exp':'UMIs_MPRA_BC'},axis=1)
df

,cellBC,CRE_id,UMIs_MPRA_BC,rep_id
0,GGGACAACAGGAGGAG,chr1_169599292_169599425_-,10,1
1,GGGACAACAGGAGGAG,chrX_55744115_55744248_+,4,1
2,GGGACAACAGGAGGAG,chr1_229569774_229569907_-,71,1
3,GGGACAACAGGAGGAG,chr17_10325195_10325328_-,11,1
4,GGGACAACAGGAGGAG,chr15_41221471_41221604_+,12,1
...,...,...,...,...
998823,TTATTGCCACAACGCC,chr8_101162797_101162930_+,1,2
998824,TTATTGCCACAACGCC,chr19_52490015_52490148_-,1,2
998825,TTATTGCCACAACGCC,chr6_42928425_42928558_+,1,2
998826,TTATTGCCACAACGCC,chr9_19380167_19380300_-,1,2


In [14]:
cell_cycle=pd.read_csv(f'{zenodo_data_root}/sc_cellBC_association/k562_cell_cycle_phase.csv')
cell_identity=pd.read_csv(f'{zenodo_data_root}/sc_cellBC_association/k562_cell_state_clustering.csv')

In [15]:
def fix(df):
    df['cellBC']=df['Unnamed: 0'].apply(lambda x : x.split("-")[0])
    df['replicate']=df['Unnamed: 0'].apply(lambda x : x.split("-")[2])
    df=df.drop('Unnamed: 0',axis=1)
    return df

In [16]:
cell_type_key=fix(cell_cycle).merge(fix(cell_identity),on=["replicate","cellBC"],how="inner")

In [17]:
f"input {len(cell_cycle)} and {len(cell_identity)} , output {len(cell_type_key)}"

'input 4276 and 4041 , output 4027'

Not bad loss.

In [18]:
cell_type_key.rename({'replicate':'rep_id'},axis=1,inplace=True)

In [19]:
cell_type_key["rep_id"]=cell_type_key["rep_id"].astype(int)

In [20]:
sensible_join=df.merge(cell_type_key,on=["rep_id","cellBC"],how="left")

In [21]:
sensible_join

,cellBC,CRE_id,UMIs_MPRA_BC,rep_id,phase,clusters
0,GGGACAACAGGAGGAG,chr1_169599292_169599425_-,10,1,NaN,NaN
1,GGGACAACAGGAGGAG,chrX_55744115_55744248_+,4,1,NaN,NaN
2,GGGACAACAGGAGGAG,chr1_229569774_229569907_-,71,1,NaN,NaN
3,GGGACAACAGGAGGAG,chr17_10325195_10325328_-,11,1,NaN,NaN
4,GGGACAACAGGAGGAG,chr15_41221471_41221604_+,12,1,NaN,NaN
...,...,...,...,...,...,...
998823,TTATTGCCACAACGCC,chr8_101162797_101162930_+,1,2,NaN,NaN
998824,TTATTGCCACAACGCC,chr19_52490015_52490148_-,1,2,NaN,NaN
998825,TTATTGCCACAACGCC,chr6_42928425_42928558_+,1,2,NaN,NaN
998826,TTATTGCCACAACGCC,chr9_19380167_19380300_-,1,2,NaN,NaN


In [22]:
f"input {len(df)} and {len(cell_type_key)} , output {len(sensible_join)}"

'input 998828 and 4027 , output 998828'

In [23]:
sensible_join.isna().any(axis=1).sum()


np.int64(607863)

In [24]:
sensible_join[sensible_join.isna().any(axis=1)]

,cellBC,CRE_id,UMIs_MPRA_BC,rep_id,phase,clusters
0,GGGACAACAGGAGGAG,chr1_169599292_169599425_-,10,1,NaN,NaN
1,GGGACAACAGGAGGAG,chrX_55744115_55744248_+,4,1,NaN,NaN
2,GGGACAACAGGAGGAG,chr1_229569774_229569907_-,71,1,NaN,NaN
3,GGGACAACAGGAGGAG,chr17_10325195_10325328_-,11,1,NaN,NaN
4,GGGACAACAGGAGGAG,chr15_41221471_41221604_+,12,1,NaN,NaN
...,...,...,...,...,...,...
998823,TTATTGCCACAACGCC,chr8_101162797_101162930_+,1,2,NaN,NaN
998824,TTATTGCCACAACGCC,chr19_52490015_52490148_-,1,2,NaN,NaN
998825,TTATTGCCACAACGCC,chr6_42928425_42928558_+,1,2,NaN,NaN
998826,TTATTGCCACAACGCC,chr9_19380167_19380300_-,1,2,NaN,NaN


In [35]:
print(len(set(cell_type_key["cellBC"])))
print(len(set(df["cellBC"])))
len(set(cell_type_key["cellBC"]).intersection(set(df["cellBC"])))

4027
3383


2718

Ok. only ~3k cell overlap. Compared to manuscript claim of "4,041 cells from two biological replicates"...

First, let's check that weird combo file I found previously. 

In [37]:
print(len(set(mystery["cellBC"])))
print(len((set(df["cellBC"]))))
print(len(set(mystery["cellBC"]).intersection(set(df["cellBC"]))))

3383
3383
3383


ok, so no help there. Seem to just be the same files. 

Let's check the ncbi files and see if they are the same? Since in the cohen mixed cell they were different.

In [38]:
import urllib.request

rep1_ncbi_k562_path=f"{data_root}/GSM5687485_k562_pBC_exp_rep1.csv.gz"

urllib.request.urlretrieve("https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM5687485&format=file&file=GSM5687485%5Fk562%5FpBC%5Fexp%5Frep1%2Ecsv%2Egz",
                           rep1_ncbi_k562_path)

rep2_ncbi_k562_path=f"{data_root}/GSM5687485_k562_pBC_exp_rep2.csv.gz"


urllib.request.urlretrieve("https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM5687486&format=file&file=GSM5687486%5Fk562%5FpBC%5Fexp%5Frep2%2Ecsv%2Egz",
                           rep2_ncbi_k562_path)

('/home/mcn26/palmer_scratch/tabula_data/GSM5687485_k562_pBC_exp_rep2.csv.gz',
 <http.client.HTTPMessage at 0x15192c7e4d60>)

In [46]:
rep1_ncbi=pd.read_csv(rep1_ncbi_k562_path)
rep2_ncbi=pd.read_csv(rep2_ncbi_k562_path)

In [49]:
all(rep1_ncbi==rep1) and all(rep2_ncbi==rep2)

True

In [ ]:
#cell_cycle
#cell_identity

I guess I have to believe it then...? Will avoid using for now, espc since I can't independently confirm cell cycle and identity info.